In [ ]:
import os
import yaml

import foolbox as fb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

from model.VGG19 import VGG19
from preprocess.preprocess import load_data

# Config

In [ ]:
with open('config/config.yml', 'r') as stream:
    config = yaml.safe_load(stream)

ROOT_DIRECTORY = os.path.dirname(os.path.abspath('__file__'))

# Load Data

In [ ]:
path_to_set = os.path.join(ROOT_DIRECTORY, config["path_to_data"])
path_to_test_csv = os.path.join(ROOT_DIRECTORY, config["path_to_test_csv"])
(X_test, y_test) = load_data(path_to_test_csv, path_to_set, config["img_width"], config["img_height"])
X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size = 0.5, random_state = 0)

# Normalize Data

In [ ]:
X_test = X_test.astype("float32") / 255.0

# Load weights of Models

In [ ]:
input_shape = (config["img_height"], config["img_width"], 3)
name_of_models = []
# Single Step
path_to_single_step_attacks = os.path.join(ROOT_DIRECTORY, "../weights2")
single_steps = ['DDN_alpha=1_N=1', 'FGM_alpha=1_N=1', 'FGSM_alpha=1_N=1', 'R+FGSM_alpha=1_N=1']
for att in single_steps:
    name_of_models.append(os.path.join(path_to_single_step_attacks, "{}_weight_a.h5".format(att)))
    name_of_models.append(os.path.join(path_to_single_step_attacks, "{}_weight_b.h5".format(att)))

# Multi Step
path_to_multi_step_attacks = os.path.join(ROOT_DIRECTORY, "../weights3")
multi_steps = ['PGD1_alpha=1_N=1', 'PGD2_alpha=1_N=1']
for att in multi_steps:
    name_of_models.append(os.path.join(path_to_multi_step_attacks, "{}_weight_a.h5".format(att)))
    name_of_models.append(os.path.join(path_to_multi_step_attacks, "{}_weight_b.h5".format(att)))

# Misc
path_to_misc_attacks = os.path.join(ROOT_DIRECTORY, "../weights4")
misc_attacks = ['GaussianAttack_alpha=1_N=1', 'InversionAttack_alpha=1_N=1', 'UniformAttack_alpha=1_N=1']
for att in misc_attacks:
    name_of_models.append(os.path.join(path_to_misc_attacks, "{}_weight_a.h5".format(att)))
    name_of_models.append(os.path.join(path_to_misc_attacks, "{}_weight_b.h5".format(att)))

name_of_models = sorted(name_of_models)
models = []

for name in name_of_models:
    model = VGG19(input_shape = input_shape, num_classes = config["num_classes"])
    model.load_weights(name)
    models.append(model)

# Compile Models

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = config["learning_rate"], decay = config["learning_rate"] / (config["epochs"]))
loss = config["loss_function"]
metrics = config["metrics"]
for model in models:
    model.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Test transferability

# Using foolbox

In [ ]:
preprocessing = dict()
bounds = (0, 1)
fmodels = []

for model in models:
    fmodels.append(fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing))

X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

# Check if models loaded properly
for i in range(len(fmodels)):
    name = os.path.basename(name_of_models[i])
    print('Model {}: {}'.format(name, fb.utils.accuracy(fmodels[i], X_test, y_test)))

# FGSM

In [ ]:
N = len(fmodels)

In [ ]:
attack = fb.attacks.FGSM()
eps = 0.031
total_acc = 0
metric2_score = 0
pair_count = 0

In [ ]:
for i in range(N):
    name = os.path.basename(name_of_models[i])
    raw, adversarial_images, is_adv = attack(fmodels[i], X_test, y_test, epsilons = eps)
    print('Using {} as attacker'.format(name))
    for j in range(N):
        if i == j:
            continue
        # Metric 2
        indexes_of_wrong_images = is_adv
        adversarial_images = adversarial_images
        wrong_classified_images = adversarial_images[indexes_of_wrong_images]
        metric2_score += fb.utils.accuracy(fmodels[j], wrong_classified_images, y_test[indexes_of_wrong_images])

        # Metric 1
        total_acc += fb.utils.accuracy(fmodels[j], adversarial_images, y_test)

        pair_count += 1

In [ ]:
print(total_acc / pair_count)

In [ ]:
print(metric2_score / pair_count)

# MISC

In [ ]:
from foolbox.distances import LpDistance

attack = fb.attacks.L2ClippingAwareAdditiveGaussianNoiseAttack()
eps = 0.031
total_acc = 0
metric2_score = 0
pair_count = 0

In [ ]:
for i in range(N):
    name = os.path.basename(name_of_models[i])
    raw, adversarial_images, is_adv = attack(fmodels[i], X_test, y_test, epsilons = eps)
    print('Using {} as attacker'.format(name))
    for j in range(N):
        if i == j:
            continue
        # Metric 2
        indexes_of_wrong_images = is_adv
        adversarial_images = adversarial_images
        wrong_classified_images = adversarial_images[indexes_of_wrong_images]
        metric2_score += fb.utils.accuracy(fmodels[j], wrong_classified_images, y_test[indexes_of_wrong_images])

        # Metric 1
        total_acc += fb.utils.accuracy(fmodels[j], adversarial_images, y_test)

        pair_count += 1

In [ ]:
print(total_acc / pair_count)

In [ ]:
print(metric2_score / pair_count)

# FGM

In [ ]:
attack = fb.attacks.FGM()
eps = 1
total_acc = 0
metric2_score = 0
pair_count = 0

In [ ]:
for i in range(N):
    name = os.path.basename(name_of_models[i])
    raw, adversarial_images, is_adv = attack(fmodels[i], X_test, y_test, epsilons = eps)
    print('Using {} as attacker'.format(name))
    for j in range(N):
        if i == j:
            continue
        # Metric 1
        total_acc += fb.utils.accuracy(fmodels[j], adversarial_images, y_test)

        # Metric 2
        indexes_of_wrong_images = is_adv
        adversarial_images = adversarial_images
        wrong_classified_images = adversarial_images[indexes_of_wrong_images]
        metric2_score += fb.utils.accuracy(fmodels[j], wrong_classified_images, y_test[indexes_of_wrong_images])

        pair_count += 1

In [ ]:
print(total_acc / pair_count)

In [ ]:
print(metric2_score / pair_count)